In [1]:
import pandas as pd
import boto3
import json
import sys
import configparser

In [2]:

config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")
DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")

DB_NAME                = config.get("DB","DB_NAME")
DB_USER                = config.get("DB","DB_USER")
DB_PASSWORD            = config.get("DB","DB_PASSWORD")
DB_PORT                = config.get("DB","DB_PORT")

IAM_ROLE_NAME      = config.get("IAM_ROLE", "ARN")

(DB_USER, DB_PASSWORD, DB_NAME)

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DB_NAME", "DB_USER", "DB_PASSWORD", "DB_PORT", "IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DB_NAME, DB_USER, DB_PASSWORD, DB_PORT, IAM_ROLE_NAME]
             })

,Param,Value
0,DWH_CLUSTER_TYPE,multi-node
1,DWH_NUM_NODES,1
2,DWH_NODE_TYPE,dc2.large
3,DWH_CLUSTER_IDENTIFIER,dwhCluster
4,DB_NAME,dwh
5,DB_USER,dwhuser
6,DB_PASSWORD,Passw0rd
7,DB_PORT,5439
8,IAM_ROLE_NAME,'arn:aws:iam::591590737336:role/myRedshiftRole'


In [5]:
import boto3

# ec2 = boto3.resource('ec2',
#                        region_name="us-east-2",
#                        aws_access_key_id=KEY,
#                        aws_secret_access_key=SECRET
#                     )

# s3 = boto3.resource('s3', region_name="us-east-2")
# s3 = boto3.client('s3',
#                        region_name="us-east-2",
#                        aws_access_key_id=KEY,
#                        aws_secret_access_key=SECRET
#                    )

s3 = boto3.resource('s3',
                       region_name="us-east-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )

iam = boto3.client('iam',
                   aws_access_key_id=KEY,
                   aws_secret_access_key=SECRET,
                   region_name='us-east-2')

redshift = boto3.client('redshift',
                       region_name="us-east-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

In [7]:
obj = s3.Bucket('udacity-dend')

for i in obj.objects.filter(Prefix='log-data/'):
    print(i)

s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-01-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-02-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-03-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-04-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-05-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-06-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-07-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-08-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-09-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-10-events.json')
s3.ObjectSummary(b

In [8]:
obj = s3.get_object(Bucket="udacity-dend",Key="log-data/2018/11/2018-11-09-events.json")


my_json = obj['Body'].read().decode("utf-8")


len(my_json.split("}"))
l = my_json.split("}")

for i in range(len(l)):
    l[i] = l[i]+'}'
    

j1 = json.loads(l[0])
print(j1)


AttributeError: 's3.ServiceResource' object has no attribute 'get_object'

In [11]:
obj_song = s3.get_object(Bucket="udacity-dend",Key="song-data/A/A/A/TRAAAAK128F9318786.json")

my_json_song = obj_song['Body'].read().decode("utf-8")
my_json_song

# len(my_json_song.split("}"))
# l = my_json_song.split("}")

# for i in range(len(l)):
#     l[i] = l[i]+'}'
    

# j1 = json.loads(l[0])
# print(j1)

'{"song_id": "SOBLFFE12AF72AA5BA", "num_songs": 1, "title": "Scream", "artist_name": "Adelitas Way", "artist_latitude": null, "year": 2009, "duration": 213.9424, "artist_id": "ARJNIUY12298900C91", "artist_longitude": null, "artist_location": ""}'

In [20]:
paginator = s3.get_paginator('list_objects')
operation_parameters = {'Bucket': 'udacity-dend',
                        'Prefix': 'log-data/'}

page_iterator = paginator.paginate(**operation_parameters)
count=0
for page in page_iterator:
    if count==0:
        print(page['Contents'][0:10])
    else:
        pass
    count+=1

[{'Key': 'log-data/', 'LastModified': datetime.datetime(2019, 4, 7, 3, 19, 23, tzinfo=tzlocal()), 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"', 'Size': 0, 'StorageClass': 'STANDARD'}, {'Key': 'log-data/2018/11/2018-11-01-events.json', 'LastModified': datetime.datetime(2019, 4, 7, 3, 19, 44, tzinfo=tzlocal()), 'ETag': '"21bae37b41c56b66973312a07322b5e4"', 'Size': 7151, 'StorageClass': 'STANDARD'}, {'Key': 'log-data/2018/11/2018-11-02-events.json', 'LastModified': datetime.datetime(2019, 4, 7, 3, 19, 44, tzinfo=tzlocal()), 'ETag': '"c726b249410a532cce10fa166ce8616c"', 'Size': 83585, 'StorageClass': 'STANDARD'}, {'Key': 'log-data/2018/11/2018-11-03-events.json', 'LastModified': datetime.datetime(2019, 4, 7, 3, 19, 45, tzinfo=tzlocal()), 'ETag': '"696af259d3203446b846de7937b5810c"', 'Size': 54084, 'StorageClass': 'STANDARD'}, {'Key': 'log-data/2018/11/2018-11-04-events.json', 'LastModified': datetime.datetime(2019, 4, 7, 3, 19, 45, tzinfo=tzlocal()), 'ETag': '"3d10cad17e2279b29da00756a8856

In [9]:
paginator = s3.get_paginator('list_objects')
operation_parameters = {'Bucket': 'udacity-dend',
                        'Prefix': 'song-data/'}

page_iterator = paginator.paginate(**operation_parameters)
count=0
for page in page_iterator:
    if count==0:
        print(page['Contents'][0:10])
    else:
        pass
    count+=1

[{'Key': 'song-data/', 'LastModified': datetime.datetime(2019, 4, 5, 0, 9, 20, tzinfo=tzlocal()), 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"', 'Size': 0, 'StorageClass': 'STANDARD'}, {'Key': 'song-data/A/A/A/TRAAAAK128F9318786.json', 'LastModified': datetime.datetime(2019, 4, 5, 0, 10, 3, tzinfo=tzlocal()), 'ETag': '"07d807ffda6d57023dd98bb229304fd0"', 'Size': 244, 'StorageClass': 'STANDARD'}, {'Key': 'song-data/A/A/A/TRAAAAV128F421A322.json', 'LastModified': datetime.datetime(2019, 4, 5, 0, 10, 3, tzinfo=tzlocal()), 'ETag': '"223beb325a8f435a95d5c9aeda774719"', 'Size': 303, 'StorageClass': 'STANDARD'}, {'Key': 'song-data/A/A/A/TRAAABD128F429CF47.json', 'LastModified': datetime.datetime(2019, 4, 5, 0, 10, 3, tzinfo=tzlocal()), 'ETag': '"d522f25a4afaeb8204ec110e9429da34"', 'Size': 268, 'StorageClass': 'STANDARD'}, {'Key': 'song-data/A/A/A/TRAAACN128F9355673.json', 'LastModified': datetime.datetime(2019, 4, 5, 0, 10, 3, tzinfo=tzlocal()), 'ETag': '"b05e400ecc81045e52cd17646342a648"', 'S

KeyboardInterrupt: 

In [ ]:
config = configparser.ConfigParser()
    config.read('dwh.cfg')

    conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
    cur = conn.cursor()


In [5]:
print(config.get("S3","LOG_DATA"))

's3://udacity-dend/log_data'
